<center>
<img src="https://laelgelcpublic.s3.sa-east-1.amazonaws.com/lael_50_years_narrow_white.png.no_years.400px_96dpi.png" width="300" alt="LAEL 50 years logo">
<h3>APPLIED LINGUISTICS GRADUATE PROGRAMME (LAEL)</h3>
</center>
<hr>

# Corpus Linguistics - Study 1 - Phase 1 - Mariana

## Prerequisites

Make sure the prerequisites in [CL_LMDA_prerequisites](https://github.com/laelgelc/laelgelc/blob/main/CL_LMDA_prerequisites.ipynb) are satisfied.

## Dataset

Please download the following dataset (Right-click on the link and choose `Open link in a new tab` to download the corresponding file):
- [mari2016_2022_pt.jsonl](https://pucsp-my.sharepoint.com/:u:/g/personal/ra00341729_pucsp_edu_br/EfSI2XLs9AlKqSkqJUW8TNIBEd_gXpNgxQX7qb7XI7T8Mw?e=iMGZTD)

## Importing the required libraries

In [1]:
import pandas as pd
import demoji
import re
import os
from collections import Counter

## Data wrangling

### Importing the tweet raw data into a dataframe

In [2]:
df_tweets_raw_data = pd.read_json('mari2016_2022_pt.jsonl', lines=True)

In [3]:
df_tweets_raw_data.head(5)

created_at   author_id         username  \
0 2016-10-03 15:00:41  1090552219  PopSeriesBrasil   
1 2016-10-03 15:00:54  2955241713         crl_jana   
2 2016-10-30 15:00:02   324548290      pxthifraga_   
3 2016-10-13 15:20:17  2751208197      heeyvasques   
4 2016-10-13 15:20:25  4158067325     Fuckinshadyy   

                                           tweet_url  \
0  https://twitter.com/PopSeriesBrasil/status/782...   
1  https://twitter.com/crl_jana/status/7829586130...   
2  https://twitter.com/pxthifraga_/status/7927428...   
3  https://twitter.com/heeyvasques/status/7865873...   
4  https://twitter.com/Fuckinshadyy/status/786587...   

                                                text  
0  Sheldon com problemas no novo episódio de #TBB...  
1  "Pra vcs, qual a causa de violência?" "inveja ...  
2  Fui votar e esqueci a identidade,já tava no me...  
3  pareço até stalker da isabella mas não sou nao...  
4            oi crianças, vocês gostam de violência?

In [4]:
df_tweets_raw_data.shape

(5409904, 5)

#### Inspecting a few tweets

In [5]:
inspected_row = 1
print('username:' + df_tweets_raw_data.loc[inspected_row, 'username'])
print('text:' + df_tweets_raw_data.loc[inspected_row, 'text'])
print('tweet_url:' + df_tweets_raw_data.loc[inspected_row, 'tweet_url'])

username:crl_jana
text:"Pra vcs, qual a causa de violência?" "inveja professor" 😂😂😂
tweet_url:https://twitter.com/crl_jana/status/782958613010800640


### Inspecting the dataset and eliminating malformed data

#### Checking if data types are consistent

In [6]:
df_tweets_raw_data.dtypes

created_at    datetime64[ns]
author_id              int64
username              object
tweet_url             object
text                  object
dtype: object

#### Identifying rows that are empty in column `text`

In [7]:
print(df_tweets_raw_data['text'].isnull().sum())

0


In [8]:
df_tweets_raw_data[df_tweets_raw_data['text'].isnull()]

Empty DataFrame
Columns: [created_at, author_id, username, tweet_url, text]
Index: []

#### Dropping the rows that are empty in the column `text`

In [9]:
# Drop the rows whose column 'text' is NaN
df_tweets_raw_data = df_tweets_raw_data.dropna(subset=['text'])

# Reset the index
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)

In [10]:
print(df_tweets_raw_data['text'].isnull().sum())

0


#### Removing specific Unicode characters

The dataset may need to be cleaned of invisible Unicode characters.

##### Detecting `U+2066` and `U+2069` characters

- [U+2066](https://www.compart.com/en/unicode/U+2066)
- [U+2069](https://www.compart.com/en/unicode/U+2069)

Please refer to:
- [Python RegEx](https://www.w3schools.com/python/python_regex.asp)
- [regex101](https://regex101.com/)
- [RegExr](https://regexr.com/)

In [11]:
# Defining a function to detect specific Unicode characters
def extract_unicode_characters(df, column_name):
    unicode_chars = Counter()  # Initialize a Counter to store Unicode character counts

    for value in df[column_name]:
        if isinstance(value, str):
            # Use RegEx to find non-ASCII characters (Unicode)
#            non_ascii_chars = re.findall(r'[^\x00-\x7F]+', value)
            # Use RegEx to find specific Unicode characters - adjust the expression accordingly
            specific_unicode_chars = re.findall(r'[\u2066\u2069]', value)
            unicode_chars.update(specific_unicode_chars)

    return unicode_chars

# Inspect the dataframe for specific Unicode characters
unicode_counts = extract_unicode_characters(df_tweets_raw_data, 'text')

# Print the results
for char, count in unicode_counts.items():
    print(f'Character {char}: Count = {count}')

Character ⁦: Count = 1436
Character ⁩: Count = 1279


##### Removing `U+2066` and `U+2069` characters

In [12]:
# Defining a function to remove specific Unicode characters
def remove_specific_unicode(input_line):
    # Using RegEx to replace specific Unicode characters - adjust the expression accordingly
    cleaned_line = re.sub(r'[\u2066\u2069]', '', input_line)
    return cleaned_line

# Removing specific Unicode characters
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_specific_unicode)

#### Replacing the `LF` character by a space

Some tweets, especially the retweeted ones, contain multiple lines of text.

In [13]:
# Defining a function to replace the `LF` character by a space
def remove_cr_lf(input_line):
    # Using RegEx to replace LF by a space
    cleaned_line = re.sub(r'\n', ' ', input_line)
    return cleaned_line

# Applying the function to the 'text' column in your DataFrame
df_tweets_raw_data['text'] = df_tweets_raw_data['text'].apply(remove_cr_lf)

In [14]:
df_tweets_raw_data

created_at            author_id         username  \
0       2016-10-03 15:00:41           1090552219  PopSeriesBrasil   
1       2016-10-03 15:00:54           2955241713         crl_jana   
2       2016-10-30 15:00:02            324548290      pxthifraga_   
3       2016-10-13 15:20:17           2751208197      heeyvasques   
4       2016-10-13 15:20:25           4158067325     Fuckinshadyy   
...                     ...                  ...              ...   
5409899 2022-09-21 17:46:51  1571649879302176768   ely_amaislinda   
5409900 2022-09-07 21:53:44            163277866       andrepreis   
5409901 2022-09-13 20:03:13  1023955089604657152      MenecasAlda   
5409902 2022-09-22 19:27:51  1091197495458308096    Danizinha4321   
5409903 2022-09-06 18:12:49  1290627563648425984         mafair12   

                                                 tweet_url  \
0        https://twitter.com/PopSeriesBrasil/status/782...   
1        https://twitter.com/crl_jana/status/7829586130...   
2        https://twitter.com/pxthifraga_/status/7927428...   
3        https://twitter.com/heeyvasques/status/7865873...   
4        https://twitter.com/Fuckinshadyy/status/786587...   
...                                                    ...   
5409899  https://twitter.com/ely_amaislinda/status/1572...   
5409900  https://twitter.com/andrepreis/status/15676322...   
5409901  https://twitter.com/MenecasAlda/status/1569778...   
5409902  https://twitter.com/Danizinha4321/status/15730...   
5409903  https://twitter.com/mafair12/status/1567214251...   

                                                      text  
0        Sheldon com problemas no novo episódio de #TBB...  
1        "Pra vcs, qual a causa de violência?" "inveja ...  
2        Fui votar e esqueci a identidade,já tava no me...  
3        pareço até stalker da isabella mas não sou nao...  
4                  oi crianças, vocês gostam de violência?  
...                                                    ...  
5409899  RT @isahmkkj: botar o fone no máximo pra não l...  
5409900  @bernardokuster2 O maior problema é q os votos...  
5409901  RT @Joaquim63301491: @LulaOficial @rai10oficia...  
5409902  RT @bibiperigosaa7_: iludida é a mulher que ac...  
5409903  RT @MinLuizRamos: “Vamos preservar o que DEUS ...  

[5409904 rows x 5 columns]

### Dropping duplicates

#### Retweets

Retweets bear the RegEx pattern `'\bRT @\w+\s*:'gm` or `'\bRT @\w+\s':"gm` at the beginning of the column `text`. They should be often dropped because they are duplicates of the original tweets. In the case of this study, though, the dataset may not include the original tweets because it is a 1% random sample. Therefore, only the first occurrence of the retweet is being kept.

In [15]:
# Create a new column 'no_retweet' containing the contents of the column 'text' without any preceding 'RT @mentions:'
df_tweets_raw_data['no_retweet'] = df_tweets_raw_data['text'].str.replace(r'\bRT @\w+\s*:|\brt @\w+\s*:', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_raw_data.drop_duplicates(subset='no_retweet', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(3032586, 6)

#### Duplicate tweets

The dataset was build in a way that if a certain tweet had more than one photo, one copy of the tweet was included per unique photo. Since we are concerned with analysing just the text, those duplicates should be removed. Tweets that bear the same 'tweet_url' are duplicates - we are going to keep only the first.

In [16]:
df_tweets_raw_data.drop_duplicates(subset='tweet_url', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(3032586, 6)

#### @mentioned tweets

A few users @mention copies of tweets towards other specific users creating multiple copies of the same tweet - those duplicates should be removed.

In [17]:
# Create a new column 'no_mention' containing the contents of the column 'text' without any preceding @mentions
df_tweets_raw_data['no_mention'] = df_tweets_raw_data['text'].str.replace(r'@\w+\s*', '', regex=True)

# Drop duplicate rows except the first occurrence based on 'no_mention'
df_tweets_raw_data.drop_duplicates(subset='no_mention', keep='first', inplace=True)
df_tweets_raw_data = df_tweets_raw_data.reset_index(drop=True)
df_tweets_raw_data.shape

(3012003, 7)

## Sampling the raw data according to filtering expressions

In [18]:
# Defining the filtering expressions
#filter_words = ['venezuelano', 'venezuelana', 'aporofobia', 'crise', 'discriminação', 'estigma', 'xenofobia', 'roraizuela', 'preconceito']
filter_words = ['venezuelano', 'venezuelana', 'aporofobia', 'discriminação', 'estigma', 'xenofobia', 'roraizuela']

# Creating a boolean mask for filtering
mask = df_tweets_raw_data['text'].str.contains('|'.join(filter_words), case=False)

# Applying the mask to create 'df_tweets_filtered'
df_tweets_filtered = df_tweets_raw_data[mask]
df_tweets_filtered = df_tweets_filtered.reset_index(drop=True)

In [19]:
df_tweets_filtered

created_at            author_id         username  \
0     2016-10-13 15:01:06           2316329808         SrtaXiss   
1     2016-10-14 14:05:00             19065418      juliosevero   
2     2016-10-13 12:37:35           4876348647         ins_ana_   
3     2016-10-25 13:00:11           2858025838  ireneravachero3   
4     2016-10-13 16:01:26           2538490328  samurai_of_LORD   
...                   ...                  ...              ...   
49491 2022-09-16 21:21:15  1557186008810196992        oliviarf0   
49492 2022-09-21 04:22:29  1400181756897148928  LucasStellato10   
49493 2022-09-21 05:27:37  1247249233918545920        HBiancaVs   
49494 2022-09-21 04:00:12  1182760455988764672     ericacomentx   
49495 2022-09-21 17:23:50  1512530391550279680  Vivianemanaia06   

                                               tweet_url  \
0      https://twitter.com/SrtaXiss/status/7865825419...   
1      https://twitter.com/juliosevero/status/7869308...   
2      https://twitter.com/ins_ana_/status/7865464248...   
3      https://twitter.com/ireneravachero3/status/790...   
4      https://twitter.com/samurai_of_LORD/status/786...   
...                                                  ...   
49491  https://twitter.com/oliviarf0/status/157088555...   
49492  https://twitter.com/LucasStellato10/status/157...   
49493  https://twitter.com/HBiancaVs/status/157245750...   
49494  https://twitter.com/ericacomentx/status/157243...   
49495  https://twitter.com/Vivianemanaia06/status/157...   

                                                    text  \
0      RT @BR_DeTodos200Mi: Grupo distribui comida pa...   
1      ONU nomeia primeiro investigador de “discrimin...   
2      RT @correio_dopovo: Roraima prepara gabinete d...   
3      E quanto a situação dos Venezuelanos,silencio ...   
4      RT @juliosevero: ONU nomeia primeiro investiga...   
...                                                  ...   
49491  RT @geovana73278250: comeca logo copa p começa...   
49492  RT @duartecomenta: Dona deolane fazendo XENOFO...   
49493  RT @oppscausey: Homofobia ao vivo Xenofobia ac...   
49494  enfiem esse negócio de xenofobia no rabo e esp...   
49495  @Lucas_Bertoldo Xenofobia ? Ela é pernambucano...   

                                              no_retweet  \
0       Grupo distribui comida para venezuelanos nas ...   
1      ONU nomeia primeiro investigador de “discrimin...   
2       Roraima prepara gabinete de emergência para c...   
3      E quanto a situação dos Venezuelanos,silencio ...   
4       ONU nomeia primeiro investigador de “discrimi...   
...                                                  ...   
49491     comeca logo copa p começar a xenofobia logo😈😈😈   
49492   Dona deolane fazendo XENOFOBIA. Será que vão ...   
49493   Homofobia ao vivo Xenofobia acontecendo  E un...   
49494  enfiem esse negócio de xenofobia no rabo e esp...   
49495  @Lucas_Bertoldo Xenofobia ? Ela é pernambucano...   

                                              no_mention  
0      RT : Grupo distribui comida para venezuelanos ...  
1      ONU nomeia primeiro investigador de “discrimin...  
2      RT : Roraima prepara gabinete de emergência pa...  
3      E quanto a situação dos Venezuelanos,silencio ...  
4      RT : ONU nomeia primeiro investigador de “disc...  
...                                                  ...  
49491  RT : comeca logo copa p começar a xenofobia lo...  
49492  RT : Dona deolane fazendo XENOFOBIA. Será que ...  
49493  RT : Homofobia ao vivo Xenofobia acontecendo  ...  
49494  enfiem esse negócio de xenofobia no rabo e esp...  
49495  Xenofobia ? Ela é pernambucano gente...Aí tbm ...  

[49496 rows x 7 columns]

### Exporting the filtered data into a file for inspection

In [20]:
df_tweets_filtered.to_csv('tweets_emojified.tsv', sep='\t', index=False)

The original set of filter words resulted in 496,811 tweets. According to Professor Tony, the recommended number of texts should fall between 20 and 50 thousand tweets. Inspecting the file `tweets_emojified.tsv` as follows, it has been determined that the words 'crise' and 'preconceito' are responsible for the majority of the selected tweets. Therefore, it has been decided to remove them.

## Replacing emojis

### Demojifying the column `text`

In [21]:
# Defining a function to demojify a string
def demojify_line(input_line):
    demojified_line = demoji.replace_with_desc(input_line, sep='<em>')
    return demojified_line

df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(demojify_line)

#### Exporting the filtered data into a file for inspection

In [22]:
df_tweets_filtered.to_csv('tweets_demojified1.tsv', sep='\t', index=False)

### Separating the demojified strings with spaces

In [23]:
# Defining a function to separate the demojified strings with spaces
def preprocess_line(input_line):
    # Add a space before the first delimiter '<em>', if it is not already preceded by one
    preprocessed_line = re.sub(r'(?<! )<em>', ' <em>', input_line)
    # Add a space after the first delimiter '<em>', if it is not already followed by one
    preprocessed_line = re.sub(r'<em>(?! )', '<em> ', preprocessed_line)
    return preprocessed_line

# Separating the demojified strings with spaces
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(preprocess_line)

#### Exporting the filtered data into a file for inspection

In [24]:
df_tweets_filtered.to_csv('tweets_demojified2.tsv', sep='\t', index=False)

### Formatting the demojified strings

In [25]:
# Defining a function to format the demojified string
def format_demojified_string(input_line):
    # Defining a function to format the demojified string using RegEx
    def process_demojified_string(s):
            # Lowercase the string
            s = s.lower()
            # Replace spaces and colons followed by a space with underscores
            s = re.sub(r'(: )| ', '_', s)
            # Add the appropriate prefixes and suffixes
            s = f'EMOJI{s}e'
            return s

    # Use RegEx to find and process each demojified string
    processed_line = re.sub(r'<em>(.*?)<em>', lambda match: process_demojified_string(match.group(1)), input_line)
    return processed_line

# Formatting the demojified strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(format_demojified_string)

### Replacing the `pipe` character by the `-` character in the `text` column

Further on, a few columns of the dataframe are going to be exported into the file `tweets.txt` whose columns need to be delimited by the `pipe` character. Therefore, it is recommended that any occurrences of the `pipe` character in the `text` column are replaced by another character.

In [26]:
# Defining a function to replace the 'pipe' character by the '-' character
def replace_pipe_with_hyphen(input_string):
    modified_string = re.sub(r'\|', '-', input_string)
    return modified_string

# Replacing the 'pipe' character by the '-' character
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(replace_pipe_with_hyphen)


#### Exporting the filtered data into a file for inspection

In [27]:
df_tweets_filtered.to_csv('tweets_demojified3.tsv', sep='\t', index=False)

## Tokenising

Please refer to [What is tokenization in NLP?](https://www.analyticsvidhya.com/blog/2020/05/what-is-tokenization-nlp/).

In [28]:
# Defining a function to tokenise a string
def tokenise_string(input_line):
    # Replace URLs with placeholders
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\b'
    placeholder = '<URL>'  # Choose a unique placeholder
    urls = re.findall(url_pattern, input_line)
    tokenised_line = re.sub(url_pattern, placeholder, input_line)  # Replace URLs with placeholders
    
    # Replace curly quotes with straight ones
    tokenised_line = tokenised_line.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'")
    # Separate common punctuation marks with spaces
    tokenised_line = re.sub(r'([.\!?,"\'/()])', r' \1 ', tokenised_line)
    # Add a space before '#'
    tokenised_line = re.sub(r'(?<!\s)#', r' #', tokenised_line)  # Add a space before '#' if it is not already preceded by one
    # Reduce extra spaces by a single space
    tokenised_line = re.sub(r'\s+', ' ', tokenised_line)
    
    # Replace the placeholders with the respective URLs
    for url in urls:
        tokenised_line = tokenised_line.replace(placeholder, url, 1)
    
    return tokenised_line

# Tokenising the strings
df_tweets_filtered['text'] = df_tweets_filtered['text'].apply(tokenise_string)

## Creating the files `file_index.txt` and `tweets.txt`

### Creating column `text_id`

In [29]:
df_tweets_filtered['text_id'] = 't' + df_tweets_filtered.index.astype(str).str.zfill(6)

### Creating column `conversation`

In [30]:
df_tweets_filtered['author_id'] = df_tweets_filtered['author_id'].astype(str)

In [31]:
df_tweets_filtered['conversation'] = 'v:' + df_tweets_filtered['author_id'].str.replace('id_', '')

### Creating column `date`

In [32]:
# Convert 'created_at' to datetime format
df_tweets_filtered['created_at'] = pd.to_datetime(df_tweets_filtered['created_at'])

# Extract the date part (without time) into a new column 'date'
df_tweets_filtered['date'] = df_tweets_filtered['created_at'].dt.date

# Add the prefix 'd:' to the 'date' values
df_tweets_filtered['date'] = 'd:' + df_tweets_filtered['date'].astype(str)

### Creating column `text_url`

In [33]:
df_tweets_filtered['text_url'] = 'url:' + df_tweets_filtered['tweet_url']

### Creating column `user`

In [34]:
df_tweets_filtered['user'] = 'u:' + df_tweets_filtered['username']

### Creating column `content`

In [35]:
df_tweets_filtered['content'] = 'c:' + df_tweets_filtered['text']

### Reordering the created columns

Please refer to:
- [Python - List Comprehension 1](https://www.w3schools.com/python/python_lists_comprehension.asp)
- [Python - List Comprehension 2](https://treyhunner.com/2015/12/python-list-comprehensions-now-in-color/)

In [36]:
# Reorder the columns (we use list comprehension to create a list of all columns except 'text_id', 'variable', 'date' and 'text_url')
df_tweets_filtered = df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url', 'user', 'content'] + [col for col in df_tweets_filtered.columns if col not in ['text_id', 'conversation', 'date', 'text_url', 'user', 'content']]]

In [37]:
df_tweets_filtered

text_id           conversation          date  \
0      t000000           v:2316329808  d:2016-10-13   
1      t000001             v:19065418  d:2016-10-14   
2      t000002           v:4876348647  d:2016-10-13   
3      t000003           v:2858025838  d:2016-10-25   
4      t000004           v:2538490328  d:2016-10-13   
...        ...                    ...           ...   
49491  t049491  v:1557186008810196992  d:2022-09-16   
49492  t049492  v:1400181756897148928  d:2022-09-21   
49493  t049493  v:1247249233918545920  d:2022-09-21   
49494  t049494  v:1182760455988764672  d:2022-09-21   
49495  t049495  v:1512530391550279680  d:2022-09-21   

                                                text_url               user  \
0      url:https://twitter.com/SrtaXiss/status/786582...         u:SrtaXiss   
1      url:https://twitter.com/juliosevero/status/786...      u:juliosevero   
2      url:https://twitter.com/ins_ana_/status/786546...         u:ins_ana_   
3      url:https://twitter.com/ireneravachero3/status...  u:ireneravachero3   
4      url:https://twitter.com/samurai_of_LORD/status...  u:samurai_of_LORD   
...                                                  ...                ...   
49491  url:https://twitter.com/oliviarf0/status/15708...        u:oliviarf0   
49492  url:https://twitter.com/LucasStellato10/status...  u:LucasStellato10   
49493  url:https://twitter.com/HBiancaVs/status/15724...        u:HBiancaVs   
49494  url:https://twitter.com/ericacomentx/status/15...     u:ericacomentx   
49495  url:https://twitter.com/Vivianemanaia06/status...  u:Vivianemanaia06   

                                                 content          created_at  \
0      c:RT @BR_DeTodos200Mi: Grupo distribui comida ... 2016-10-13 15:01:06   
1      c:ONU nomeia primeiro investigador de " discri... 2016-10-14 14:05:00   
2      c:RT @correio_dopovo: Roraima prepara gabinete... 2016-10-13 12:37:35   
3      c:E quanto a situação dos Venezuelanos , silen... 2016-10-25 13:00:11   
4      c:RT @juliosevero: ONU nomeia primeiro investi... 2016-10-13 16:01:26   
...                                                  ...                 ...   
49491  c:RT @geovana73278250: comeca logo copa p come... 2022-09-16 21:21:15   
49492  c:RT @duartecomenta: Dona deolane fazendo XENO... 2022-09-21 04:22:29   
49493  c:RT @oppscausey: Homofobia ao vivo Xenofobia ... 2022-09-21 05:27:37   
49494  c:enfiem esse negócio de xenofobia no rabo e e... 2022-09-21 04:00:12   
49495  c:@Lucas_Bertoldo Xenofobia ? Ela é pernambuca... 2022-09-21 17:23:50   

                 author_id         username  \
0               2316329808         SrtaXiss   
1                 19065418      juliosevero   
2               4876348647         ins_ana_   
3               2858025838  ireneravachero3   
4               2538490328  samurai_of_LORD   
...                    ...              ...   
49491  1557186008810196992        oliviarf0   
49492  1400181756897148928  LucasStellato10   
49493  1247249233918545920        HBiancaVs   
49494  1182760455988764672     ericacomentx   
49495  1512530391550279680  Vivianemanaia06   

                                               tweet_url  \
0      https://twitter.com/SrtaXiss/status/7865825419...   
1      https://twitter.com/juliosevero/status/7869308...   
2      https://twitter.com/ins_ana_/status/7865464248...   
3      https://twitter.com/ireneravachero3/status/790...   
4      https://twitter.com/samurai_of_LORD/status/786...   
...                                                  ...   
49491  https://twitter.com/oliviarf0/status/157088555...   
49492  https://twitter.com/LucasStellato10/status/157...   
49493  https://twitter.com/HBiancaVs/status/157245750...   
49494  https://twitter.com/ericacomentx/status/157243...   
49495  https://twitter.com/Vivianemanaia06/status/157...   

                                                    text  \
0      RT @BR_DeTodos200Mi: Grupo distribui comida pa...   
1      ONU nomeia primeiro inv

### Creating the file `file_index.txt`

In [38]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'text_url']].to_csv('file_index.txt', sep=' ', index=False, header=False, encoding='utf-8', lineterminator='\n')

### Creating the file `tweets.txt`

In [39]:
folder = 'tweets'
try:
    os.mkdir(folder)
    print(f'Folder {folder} created!')
except FileExistsError:
    print(f'Folder {folder} already exists')

Folder tweets already exists


Note: The parameters `doublequote=False` and `escapechar=' '` are required to avoid that the column content is doublequoted with '"' in sentences that use characters that need to be escaped such as double quote '"' itself - this causes a malformed response from TreeTagger.

In [40]:
df_tweets_filtered[['text_id', 'conversation', 'date', 'user', 'content']].to_csv(f'{folder}/tweets.txt', sep='|', index=False, header=False, encoding='utf-8', lineterminator='\n', doublequote=False, escapechar=' ')

## Tagging with TreeTagger

- On Visual Studio Code (VS Code), open the folder where your project is located with `Open Folder...`
- Open a WSL Ubuntu Terminal on VS Code
- **Important**: Activate the `my_env` Python environment by executing `source "$HOME"/my_env/bin/activate`
- Proceed as indicated

Purpose: Annotate the texts in `tweets/tweets.txt` with part-of-speech and lemma information.
- Input
    - `file_index.txt`
    - `tweets/tweets.txt`
- Output
    - `tweets/tagged.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash treetagging.sh
--- treetagging t000000 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000001 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000002 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
--- treetagging t000003 / t018205 ---
        reading parameters ...
        tagging ...
         finished.
<omitted>
```

## Processing `tokenstypes`

Purpose: Capture the content tokens (specific occurrences of words) and the content types (general concept of words) from `tweets/tagged.txt`.
- Input
    - `file_index.txt`
    - `tweets/tagged.txt`
- Output
    - `tweets/tokens.txt`
    - `tweets/types.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash tokenstypes.sh
--- tokenstypes t000000 / 18206 ---
--- tokenstypes t000001 / 18206 ---
--- tokenstypes t000002 / 18206 ---
--- tokenstypes t000003 / 18206 ---
--- tokenstypes t000004 / 18206 ---
--- tokenstypes t000005 / 18206 ---
<omitted>
```

## Processing `toplemmas`

Purpose: Determine the 1.000 top lemmas. **Important**: This process requires manual inspection. Non-meaningful lemmas should be excluded by updating `stoplist.sed` and reiterating the processing. Proceed as indicated in this [video tutorial](https://youtu.be/4UIPpdoozP0?si=_3md9w79njZY86PE), from 20:05 to 25:16.
- Input
    - `tweets/types.txt`
    - `stoplist.sed`: List of rules that allows the exclusion of a certain lemmas
- Output
    - `selectedwords` = `var_index.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash toplemmas.sh
```

## Processing `sas`

Purpose: Prepare input data for processing in SAS.
- Input
    - `tweets/types.txt`
    - `selectedwords`
    - `file_index.txt`
- Output
    - `columns`
    - `sas/data.txt`
    - `sas/dates.txt`
    - `sas/wcount.txt`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash sas.sh
--- v000001 ---
--- v000002 ---
--- v000003 ---
--- v000004 ---
--- v000005 ---
<omitted>
--- v001000 ---
[nltk_data] Downloading package punkt to /home/eyamrog/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Word counts written to sas/wcount.txt
```

## Processing `datamatrix`

Purpose: Prepares input data for calculating the correlation matrix.
- Input
    - `file_index.txt`
    - `columns`
    - `selectedwords`
- Output
    - `file_ids.txt`
    - `data.csv`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash datamatrix.sh
--- v000001 ---
--- v000002 ---
--- v000003 ---
--- v000004 ---
--- v000005 ---
<omitted>
--- v001000 ---
--- data.csv ...---
```

## Processing `correlationmatrix`

Purpose: Calculates the correlation matrix.
- Input
    - `data.csv`
- Output
    - `correlation`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash correlationmatrix.sh
--- python correlation ... ---
```

## Processing `formats`

Purpose: Prepare input data for processing in SAS.
- Input
    - `data.csv`
    - `selectedwords`
- Output
    - `sas/corr.txt`
    - `sas/word_labels_format.sas`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ source "$HOME"/my_env/bin/activate
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash formats.sh
--- sas/sas/corr.txt ---
--- sas/word_labels_format.sas ---
```

## Processing the statistical procedures on SAS

- Log in to your [SAS OnDemand for Academics](https://welcome.oda.sas.com/) account
- Proceed as indicated in this [video tutorial](https://youtu.be/I3u9zD3jyOA?si=68uIKVc2iusGG2KY)

## Processing `examples`

Purpose: Extract examples for analysis.
- Input
    - `sas/output_"$project"/loadtable.html`
    - `sas/output_"$project"/"$project"_scores.tsv`
    - `sas/output_"$project"/"$project"_scores_only.tsv`
- Output
    - `examples/factors`
    - `example files`

```
(my_env) eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ bash examples.sh
6780
1246
698
123
--- examples f1pos ---
--- factor 1 pos # 000001 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000002 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000003 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000004 ---
tr: warning: an unescaped backslash at end of string is not portable
--- factor 1 pos # 000005 ---
tr: warning: an unescaped backslash at end of string is not portable
<ommitted>
```

## Results

Right-click on the link and choose `Save link as` to download the corresponding file.

- [CL_St1_Mariana_Results.tar.gz](https://pucsp-my.sharepoint.com/:u:/g/personal/ra00341729_pucsp_edu_br/EU1-gvN-4pRDqIuGkPWHry4B6dJ7dY6gWqaKcydFL7evvA?e=AGpfTJ)

Note:
- Proceed as follows to extract the files from the archive `CL_St1_Mariana_Results.tar.gz`

```
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ tar xzvf CL_St1_Mariana_Results.tar.gz
<omitted>
eyamrog@Rog-ASUS:/mnt/c/Users/eyamr/Downloads$ 
```